In [ ]:
#| include: false
#| echo: false
! [ -e /content ] && pip install -Uqq gingado nbdev # install or upgrade gingado on colab

In [ ]:
#| default_exp augmentation
#| include: false
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| include: false
from nbdev.showdoc import show_doc

`gingado` provides data augmentation functionalities that can help users to augment their datasets with a time series dimension. This can be done both on a stand-alone basis as the user incorporates new data on top of the original dataset, or as part of a `scikit-learn` [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) that also includes other steps like data transformation and model estimation.

## Data augmentation with SDMX

The **S**tatistical **D**ata and **M**etadata e**X**change (SDMX) is an ISO standard comprising:

* technical standards

* statistical guidelines, including cross-domain concepts and codelists

* an IT architecture and tools

SDMX is sponsored by the Bank for International Settlements, European Central Bank, Eurostat, International Monetary Fund, Organisation for Economic Co-operation and Development, United Nations, and World Bank Group.

More information about the SDMX is available on its [webpage](http://sdmx.org).

`gingado` uses SDMX to augment user datasets through the transformer `AugmentSDMX`.

In [ ]:
#| include: false
#| export

from gingado.utils import load_SDMX_data
import numpy as np
import pandas as pd
import pandasdmx as sdmx
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_selection import VarianceThreshold

class AugmentSDMX(BaseEstimator, TransformerMixin):
    "A transformer that augments a dataset using SDMX"
    
    InputIndexMessage = "The dataset to be augmented must have a row index with the date/time information"
    def _format_string(self):
        return "%Y-%m-%d" if self.data_freq_ == 'D' else "%Y-%m" if self.data_freq_ == 'M' else "%Y"
    
    def _get_dates(self):
        fstr = self._format_string()
        return {
            "startPeriod": min(self.index_).strftime(fstr),
            "endPeriod": max(self.index_).strftime(fstr),
        }

    def _transform(self, X, training=True):
        df = load_SDMX_data(sources=self.sources, keys=self.keys_, params=self.params_, verbose=self.verbose)
        if df is None:
            return X

        if training:
            # test that the dataset `X` has the same dimension as the one
            # used during training, which is an evidence they are the same
            n_samples_in_transform, n_features_in_transform = X.shape
            if n_samples_in_transform != self.n_samples_in_ or n_features_in_transform != self.n_features_in_:
                raise ValueError("The X passed to the transform() method must be compatible with the X used by the fit() method.")
            # during testing, we don't want the possibility of a different
            # set of columns being retained by virtue of different dynamics
            # in both datasets. For example, if a feature is included in the
            # training but during the test dates the variable didn't move, it
            # should not be subject to the test below so that it is still
            # included in the fitted data.
            feat_sel = VarianceThreshold() if self.variance_threshold is None else VarianceThreshold(threshold=self.variance_threshold)
            feat_sel.fit(df)
            self.features_stay_ = df.columns[feat_sel.get_support()]
            self.features_removed_ = df.columns[~feat_sel.get_support()]
            
            df = df.iloc[:, feat_sel.get_support()]
            df.columns = feat_sel.get_feature_names_out()

            df.dropna(axis=0, how='all', inplace=True)
            df.dropna(axis=1, how='all', inplace=True)        
        
        X = pd.merge(left=X, right=df, how='left', left_index=True, right_on='TIME_PERIOD')
        if 'TIME_PERIOD' in X.columns:
            X.drop(columns='TIME_PERIOD', inplace=True)
        if self.propagate_last_known_value:
            X.fillna(method="ffill", inplace=True)
        if self.fillna is not None:
            X.fillna(self.fillna)
        if training:
            X.index = self.index_
        return X

    def __init__(self, sources={'BIS': 'WS_CBPOL_D'}, variance_threshold=None, propagate_last_known_value=True, fillna = 0, verbose=True):
        self.sources = sources
        self.variance_threshold = variance_threshold
        self.propagate_last_known_value = propagate_last_known_value
        self.fillna = fillna
        self.verbose = verbose

    def fit(
            self,
            X, # a pandas Series or DataFrame with an index of datetime type
            y:None=None # `y` is kept as argument for API consistency only
        ): # A fitted version of the same AugmentSDMX instance
        # Fits instance of AugmentSDMX to `X`, learning its time series frequency
        
        try:
            self.data_freq_ = X.index.to_series().diff().min().resolution_string
        except AttributeError:
            print(self.InputIndexMessage)
            raise
        self.index_ = X.index
        self.keys_ = {'FREQ': self.data_freq_}
        X = self._validate_data(X)

        # this variable below is only included to test for consistency \
        # if `fit` and `transform` are called separately with the same `X`
        self.n_samples_in_ = X.shape[0]

        return self

    def transform(
            self,
            X, # a pandas Series or DataFrame with an index of datetime type
            y:None=None, # `y` is kept as argument for API consistency only
            training=False # `True` is `transform` is called during training; `False` if called during testing
        ): # `X` augmented with data from SDMX
        # Transforms input dataset `X` by adding the requested data using SDMX
        check_is_fitted(self)
        self.params_ = self._get_dates()
        idx = X.index
        transf_X = self._transform(X, training=training)
        transf_X.index = idx
        return transf_X

    def fit_transform(
            self, 
            X, # # a pandas Series or DataFrame with an index of datetime type
            y:None=None # `y` is kept as argument for API consistency only
            ) -> np.ndarray: # `X` augmented with data from SDMX with the same number of samples but more columns
        # Fit to data, then transform it.
        
        self.fit(X)
        self.params_ = self._get_dates()
        return self.transform(X, training=True)

In [ ]:
show_doc(AugmentSDMX)

---

[source](https://github.com/dkgaraujo/gingado/tree/main/blob/main/gingado/augmentation.py#L16){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX

>      AugmentSDMX (sources={'BIS': 'WS_CBPOL_D'}, variance_threshold=None,
>                   propagate_last_known_value=True, fillna=0, verbose=True)

A transformer that augments a dataset using SDMX

In [ ]:
show_doc(AugmentSDMX.fit)

---

[source](https://github.com/dkgaraujo/gingado/tree/main/blob/main/gingado/augmentation.py#L77){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX.fit

>      AugmentSDMX.fit (X, y:None=None)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X |  |  | a pandas Series or DataFrame with an index of datetime type |
| y | None | None | `y` is kept as argument for API consistency only |

In [ ]:
show_doc(AugmentSDMX.transform)

---

[source](https://github.com/dkgaraujo/gingado/tree/main/blob/main/gingado/augmentation.py#L99){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX.transform

>      AugmentSDMX.transform (X, y:None=None, training=False)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X |  |  | a pandas Series or DataFrame with an index of datetime type |
| y | None | None | `y` is kept as argument for API consistency only |
| training | bool | False | `True` is `transform` is called during training; `False` if called during testing |

In [ ]:
show_doc(AugmentSDMX.fit_transform)

---

[source](https://github.com/dkgaraujo/gingado/tree/main/blob/main/gingado/augmentation.py#L113){target="_blank" style="float:right; font-size:smaller"}

### AugmentSDMX.fit_transform

>      AugmentSDMX.fit_transform (X, y:None=None)

Fit to data, then transform it.

Fits transformer to `X` and `y` with optional parameters `fit_params`
and returns a transformed version of `X`.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| X | array-like of shape (n_samples, n_features) |  | # a pandas Series or DataFrame with an index of datetime type |
| y | array-like of shape (n_samples,) or (n_samples, n_outputs),                 default=None | None | `y` is kept as argument for API consistency only |
| **Returns** | **ndarray** |  | **`X` augmented with data from SDMX with the same number of samples but more columns** |

### Compatibility with `scikit-learn`

As mentioned above, `gingado`'s transformers are built to be compatible with `scikit-learn`. The code below demonstrates this compatibility.

First, we create the example dataset. In this case, it comprises the daily foreign exchange rate of selected currencies to the Euro. The Brazilian Real (BRL) is chosen for this example as the dependent variable.

In [ ]:
from gingado.utils import load_SDMX_data, Lag
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
X = load_SDMX_data(
    sources={'ECB': 'EXR'}, 
    keys={'FREQ': 'D', 'CURRENCY': ['EUR', 'AUD', 'BRL', 'CAD', 'CHF', 'GBP', 'JPY', 'SGD', 'USD']},
    params={"startPeriod": 2003}
    )
# drop rows with empty values
X.dropna(inplace=True)
# adjust column names in this simple example for ease of understanding:
# remove parts related to source and dataflow names
X.columns = X.columns.str.replace("ECB__EXR_D__", "").str.replace("__EUR__SP00__A", "")
X = Lag(lags=1, jump=0, keep_contemporaneous_X=True).fit_transform(X)
y = X.pop('BRL')
# retain only the lagged variables in the X variable
X = X[X.columns[X.columns.str.contains('_lag_')]]

Querying data from ECB's dataflow 'EXR' - Exchange Rates...


2022-09-22 22:08:39,127 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


In [ ]:
X_train, X_test = X.iloc[:-1], X.tail(1)
y_train, y_test = y.iloc[:-1], y.tail(1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5052, 8), (5052,), (1, 8), (1,))

Next, the data augmentation object provided by `gingado` adds more data. In this case, for brevity only one dataflow from one source is listed. If users want to add more SDMX sources, simply add more keys to the dictionary. And if users want data from all dataflows from a given source provided the keys and parameters such as frequency and dates match, the value should be set to `'all'`, as in `{'ECB': ['CISS'], 'BIS': 'all'}`.

In [ ]:
test_src = {'ECB': ['CISS'], 'BIS': ['WS_CBPOL_D']}

X_train__fit_transform = AugmentSDMX(sources=test_src).fit_transform(X=X_train)
X_train__fit_then_transform = AugmentSDMX(sources=test_src).fit(X=X_train).transform(X=X_train, training=True)

assert X_train__fit_transform.shape == X_train__fit_then_transform.shape

Querying data from ECB's dataflow 'CISS' - Composite Indicator of Systemic Stress...


2022-09-22 22:08:57,676 pandasdmx.reader.sdmxml - INFO: Use supplied dsd=… argument for non–structure-specific message


Querying data from BIS's dataflow 'WS_CBPOL_D' - Policy rates daily...


`AugmentSDMX` can also be part of a `Pipeline` object, which minimises operational errors during modelling and avoids using testing data during training:

This is the dataset now after this particular augmentation:

In [ ]:
print(f"No of columns: {len(X_train__fit_transform.columns)} {X_train__fit_transform.columns}")
X_train__fit_transform

In [ ]:
from gingado.augmentation import AugmentSDMX
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
pipeline = Pipeline([
    ('augmentation', AugmentSDMX(sources={'BIS': 'WS_CBPOL_D'})),
    ('imp', IterativeImputer(max_iter=10)),
    ('forest', RandomForestRegressor())
], verbose=True)

### Tuning the data augmentation to enhance model performance

And since `AugmentSDMX` can be included in a `Pipeline`, it can also be fine-tuned by parameter search techniques (such as grid search), further helping users make the best of available data to enhance performance of their models.

In [ ]:
grid = GridSearchCV(
    estimator=pipeline,
    param_grid={
        'augmentation': ['passthrough', AugmentSDMX(sources={'ECB': 'CISS'})]
    },
    verbose=3,
    cv=TimeSeriesSplit()
    )

y_pred_grid = grid.fit(X_train, y_train).predict(X_test)

In [ ]:
grid.best_params_

In [ ]:
print(f"In this particular case, the best model was achieved by {'not ' if grid.best_params_['augmentation'] == 'passthrough' else ''}using the data augmentation.")

In [ ]:
print(f"The last value in the training dataset was {y_train.tail(1).to_numpy()}. The predicted value was {y_pred_grid}, and the actual value was {y_test.to_numpy()}.")

### Sources of data

`gingado` seeks to only lists realiable data sources by choice, with a focus on official sources. This is meant to provide users with the trust that their dataset will be complemented by reliable sources. Unfortunately, it is not possible at this stage to include *all* official sources given the substantial manual and maintenance work. `gingado` leverages the existence of the [Statistical Data and Metadata eXchange (SDMX)](https://sdmx.org), an organisation of official data sources that establishes common data and metadata formats, to download data that is relevant (and hopefully also useful) to users.

The function below from the package [simpledmx](https://github.com/dkgaraujo/simpledmx) returns a list of codes corresponding to the data sources available to provide `gingado` users with data through SDMX.

In [ ]:
from gingado.utils import list_SDMX_sources

In [ ]:
list_SDMX_sources()

You can also see what the available dataflows are. The code below returns a dictionary where each key is the code for an SDMX source, and the values associated with each key are the code and name for the respective dataflows.

In [ ]:
from gingado.utils import list_all_dataflows

In [ ]:
dflows = list_all_dataflows()
dflows

For example, the dataflows from the World Bank are:

In [ ]:
dflows['WB']

In [ ]:
#| echo: false
import nbdev; nbdev.nbdev_export()